In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import random
import warnings
from datetime import datetime
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [2]:
# Load the DataFrames
df = pd.read_csv('exports/stocks.csv')
df

,stock_name,stock_code
0,7-ELEVEN MALAYSIA HOLDINGS BERHAD,5250
1,ABF MALAYSIA BOND INDEX FUND,0800EA
2,ABLE GLOBAL BERHAD,7167
3,ABLEGROUP BERHAD,7086
4,ABM FUJIYA BERHAD,5198
5,ACE INNOVATE ASIA BERHAD,03028
6,ACME HOLDINGS BERHAD,7131
7,ACO GROUP BERHAD,0218
8,ADVANCE INFORMATION MARKETING BERHAD,0122
9,ADVANCE SYNERGY BERHAD,1481


In [3]:
# Put it into a list called stock_code
stock_code = list(df['stock_code'])
stock_code

['5250',
 '0800EA',
 '7167',
 '7086',
 '5198',
 '03028',
 '7131',
 '0218',
 '0122',
 '1481',
 '5281',
 '9148',
 '7191',
 '7146',
 '0181',
 '6599',
 '5139',
 '5185',
 '7145',
 '0258',
 '7315',
 '7078',
 '0209',
 '5238',
 '2658',
 '7609',
 '5116',
 '5115',
 '2674',
 '0079',
 '2488',
 '1163',
 '03051',
 '5269',
 '5127',
 '5293',
 '5307',
 '5120',
 '03011',
 '1015',
 '7031',
 '6351',
 '7083',
 '0048',
 '4758',
 '0226',
 '6556',
 '5082',
 '5568',
 '5088',
 '7090',
 '5015',
 '6432',
 '0119',
 '7214',
 '7181',
 '7007',
 '0038',
 '0068',
 '7722',
 '7129',
 '0159',
 '0105',
 '4057',
 '7162',
 '7054',
 '03032',
 '6399',
 '0072',
 '8176',
 '7048',
 '5130',
 '7099',
 '5302',
 '03037',
 '8885',
 '5204',
 '7579',
 '6888',
 '5106',
 '7120',
 '2305',
 '5021',
 '7005',
 '03012',
 '0098',
 '5258',
 '7251',
 '1899',
 '6602',
 '0187',
 '5190',
 '3239',
 '3395',
 '5196',
 '4219',
 '5248',
 '9814',
 '7668',
 '0263',
 '6173',
 '5932',
 '0195',
 '6998',
 '5032',
 '0179',
 '5069',
 '0168',
 '9288',
 '7036',
 '

In [4]:
div_stock_code=[]
div_PCF=[]
div_5year_growth=[]
list_url=[]

driver = webdriver.Chrome()
driver.implicitly_wait(20)

# funtion to get link
def get_link(stock_code):
    
    url='https://www.bursamarketplace.com/index.php'
    driver.get(url)
    
    #click search button
    s= driver.find_element(By.XPATH,'//*[@id="newnav-mobileSearch"]')
    driver.execute_script("arguments[0].click();",s)
    
    # input code
    driver.find_element(By.XPATH, '//*[@id="newnav-search-input"]').send_keys(stock_code)
    
    # get the link
    lnks = driver.find_element(By.ID,'url')
    links=lnks.get_attribute('href')
    return links
    
    

for s in stock_code:
    try:
        url=get_link(s)+'/financials'
        driver.get(url)
        content = driver.page_source
        soup = bs(content)
    except:
        list_url.append(s)
        print('stockcode '+ s+ ' url cannot scrap')
    
    time.sleep(2)
    
    try:
        stock_div_5year_growth = driver.find_element(By.XPATH, "//div[text()='Dividend 5 Yr. Growth Rate']/following-sibling::div")
    except:
         print('stock code :'+s+' stock_div_5year_growth cannot scrap')
    try:
        stock_div_PCF = driver.find_element(By.XPATH, "//div[text()='Curr Price/CF/Shr']/following-sibling::div")
    except:
        print('stock code :'+s+' stock_div_PCF cannot scrap')
        
  
    
    div_stock_code.append(s)
    try:
        div_5year_growth.append(stock_div_5year_growth.text)
    except:
        div_5year_growth.append(0)
    
    try:
        div_PCF.append(stock_div_PCF.text)
    except:
        div_PCF.append(0)

    # importing the random module
    sec = random.randint(3,4)

    # program scrapping waiting time
    time.sleep(sec) # Seconds

df = pd.DataFrame({'stock_code':div_stock_code, 'pcf':div_PCF,'div_5year_growth':div_5year_growth}) 
df.to_csv('Share.csv', index=False, encoding='utf-8')

stock code :0800EA stock_div_5year_growth cannot scrap
stock code :0800EA stock_div_PCF cannot scrap
stockcode 2771 url cannot scrap
stock code :2771 stock_div_5year_growth cannot scrap
stock code :2771 stock_div_PCF cannot scrap
stockcode 3484 url cannot scrap
stock code :3484 stock_div_5year_growth cannot scrap
stock code :3484 stock_div_PCF cannot scrap
stockcode 0400GB url cannot scrap
stock code :0400GB stock_div_5year_growth cannot scrap
stock code :0400GB stock_div_PCF cannot scrap
stockcode 5835 url cannot scrap
stock code :5835 stock_div_5year_growth cannot scrap
stock code :5835 stock_div_PCF cannot scrap
stock code :0820EA stock_div_5year_growth cannot scrap
stock code :0820EA stock_div_PCF cannot scrap
stockcode 0401GA url cannot scrap
stock code :0401GA stock_div_5year_growth cannot scrap
stock code :0401GA stock_div_PCF cannot scrap
stockcode 8931 url cannot scrap
stock code :8931 stock_div_5year_growth cannot scrap
stock code :8931 stock_div_PCF cannot scrap
stock code :

In [5]:
df

,stock_code,pcf,div_5year_growth
0,5250,8.000,15.300
1,0800EA,0,0
2,7167,9.238,0.000
3,7086,92.329,0.000
4,5198,12.305,0.000
5,03028,161.358,0.000
6,7131,259.348,0.000
7,0218,9.150,0.000
8,0122,66.631,0.000
9,1481,24.628,-13.954


In [6]:
# Specify the file path where you want to save the CSV file
csv_file_path = 'exports/dgpcf.csv'

# Use the to_csv() method to export the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV

print(f"DataFrame saved to {csv_file_path}")

DataFrame saved to exports/dgpcf.csv
